In [ ]:
!pip install neuralforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.6/170.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import torch
from typing import Optional, Union, Tuple


import torch.nn.functional as F
from torch.distributions import Distribution
from torch.distributions import Bernoulli, Normal, StudentT, Poisson, NegativeBinomial

from torch.distributions import constraints


from neuralforecast import NeuralForecast
from neuralforecast.models import PatchTST
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic
from neuralforecast.losses.numpy import rmse, mape
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from neuralforecast.losses.pytorch import MQLoss, DistributionLoss, MSE, MAE

In [ ]:
from numpy.random import seed
from random import randrange

In [ ]:
df = pd.read_excel("SP100_vol (clean).xlsx", index_col="Date",parse_dates=True)
columns=df.columns
df_static=pd.read_excel("Static_df.xlsx")
n_series=len(df.columns)

In [ ]:
n_inputs = [3,5,10,15,21,42,84]
hidden_size = [60,90,120,150,180]
epochs=[50,100,150,250,350,450,550,650,750]
learning_rate=[0.0005,0.0001,0.00005,0.00001]
num_lr_decays=[5,3,2,1]
scaler_type=["robust","standard",'minmax']
losses=[MSE(),MAE(),MQLoss(level=[80, 90]),DistributionLoss(distribution='StudentT', level=[80, 90])]
n_head=[6,9,12,15,18]
decoder_input_size_multiplier=[0.3,0.4,0.5,0.6,0.7]
dropout=[0,0.2,0.3,0.4]
patch_len=[4,6,8,10,12]
stride=[6,7,8,9,10]
encoder_layers = [1,2,3,4,5]

# Test

In [1]:
test_length=int(len(df['NKE'])*0.3)
df1=df.unstack().reset_index(name="Value").rename(columns={"level_0":"unique_id", "Date":"ds", "Value":"y"})

NameError: ignored

# H=1

In [ ]:
RMSE = []
QLIKE = []
MAE = []
MAPE = []
H=1

for m in range (20):
  randomseed=randrange(129228148)
  model = PatchTST(h=H, input_size=42,
               hidden_size=hidden_size[3], linear_hidden_size =int(hidden_size[3]*2),
             dropout=dropout[0],  fc_dropout=dropout[0], head_dropout=0.3,
                     patch_len=patch_len[1],stride=stride[1],
             attn_dropout=0.3, n_heads=15,
               encoder_layers=2,loss=losses[0],
               max_steps=150,
             learning_rate=learning_rate[1], num_lr_decays=num_lr_decays[1],
               val_check_steps=500, batch_size=32,
               scaler_type='standard',
                random_seed= randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=df1,val_size=H,static_df=df_static,n_windows=None, test_size=test_length-test_length%H,step_size=H)
  forecasts = forecasts.dropna()
  if "PatchTST-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["PatchTST"].values
  else:
    Y_hat=forecasts["PatchTST-median"].values
  Y_true=forecasts["y"].values
  RMSE.append(np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true)))
  QLIKE.append((np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true)))
  MAE.append(np.sum(abs(Y_true-Y_hat))/len(Y_true))
  MAPE.append(np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true))

print(f"RMSE mean: {np.mean(RMSE)} | QLIKE mean: {np.mean(QLIKE)} | MAE mean: {np.mean(MAE)} | MAPE mean: {np.mean(MAPE)}")
print(f"RMSE std: {np.std(RMSE)} | QLIKE std: {np.std(QLIKE)} | MAE std: {np.std(MAE)} | MAPE std: {np.std(MAPE)}")
from scipy.stats import anderson
print(f"RMSE: {anderson(RMSE)}")
print(f"QLIKE: {anderson(QLIKE)}")
print(f"MAE: {anderson(MAE)}")
print(f"MAPE: {anderson(MAPE)}")

df_errors = pd.DataFrame({
    'RMSE': RMSE,
    'QLIKE': QLIKE,
    'MAE': MAE,
    'MAPE': MAPE
})

df_errors.to_excel("PatchTST forecasts.xlsx")

INFO:lightning_fabric.utilities.seed:Seed set to 24811742


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE mean: 0.003867450863550404 | QLIKE mean: 0.03089340644667276 | MAE mean: 0.002451990651381411 | MAPE mean: 0.19080771201703617
RMSE std: 9.994204876963835e-06 | QLIKE std: 0.00010730696429587606 | MAE std: 7.552223046300475e-06 | MAPE std: 0.000846930306242619
RMSE: AndersonResult(statistic=0.2879273109849052, critical_values=array([0.506, 0.577, 0.692, 0.807, 0.96 ]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=0.003867450863550404, scale=1.0253837850068765e-05)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')
QLIKE: AndersonResult(statistic=0.11630489323016491, critical_values=array([0.506, 0.577, 0.692, 0.807, 0.96 ]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=0.03089340644667276, scale=0.00011009462239554338)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')
MAE: AndersonResult(statistic=0.30196025788331227

# H=5

In [ ]:
RMSE = []
QLIKE = []
MAE = []
MAPE = []
H=5

for m in range (20):
  randomseed=randrange(129228148)
  model = PatchTST(h=H, input_size=42,
               hidden_size=hidden_size[3], linear_hidden_size =int(hidden_size[3]*2),
             dropout=dropout[0],  fc_dropout=dropout[0], head_dropout=0.3,
                     patch_len=patch_len[1],stride=stride[1],
             attn_dropout=0.3, n_heads=15,
               encoder_layers=2,loss=losses[0],
               max_steps=150,
             learning_rate=learning_rate[1], num_lr_decays=num_lr_decays[1],
               val_check_steps=500, batch_size=32,
               scaler_type='standard',
                random_seed= randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=df1,val_size=H,static_df=df_static,n_windows=None, test_size=test_length-test_length%H,step_size=H)
  forecasts = forecasts.dropna()
  if "PatchTST-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["PatchTST"].values
  else:
    Y_hat=forecasts["PatchTST-median"].values
  Y_true=forecasts["y"].values
  RMSE.append(np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true)))
  QLIKE.append((np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true)))
  MAE.append(np.sum(abs(Y_true-Y_hat))/len(Y_true))
  MAPE.append(np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true))

print(f"RMSE mean: {np.mean(RMSE)} | QLIKE mean: {np.mean(QLIKE)} | MAE mean: {np.mean(MAE)} | MAPE mean: {np.mean(MAPE)}")
print(f"RMSE std: {np.std(RMSE)} | QLIKE std: {np.std(QLIKE)} | MAE std: {np.std(MAE)} | MAPE std: {np.std(MAPE)}")
from scipy.stats import anderson
print(f"RMSE: {anderson(RMSE)}")
print(f"QLIKE: {anderson(QLIKE)}")
print(f"MAE: {anderson(MAE)}")
print(f"MAPE: {anderson(MAPE)}")

df_errors = pd.DataFrame({
    'RMSE': RMSE,
    'QLIKE': QLIKE,
    'MAE': MAE,
    'MAPE': MAPE
})

df_errors.to_excel("PatchTST forecasts.xlsx")

INFO:lightning_fabric.utilities.seed:Seed set to 1428100


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE mean: 0.004670837071489431 | QLIKE mean: 0.04367601527570585 | MAE mean: 0.002945678074667103 | MAPE mean: 0.22977471500808422
RMSE std: 1.0017952843768579e-05 | QLIKE std: 0.00011010847974224319 | MAE std: 7.504740386134339e-06 | MAPE std: 0.0007694144959626845
RMSE: AndersonResult(statistic=0.23839484676898337, critical_values=array([0.506, 0.577, 0.692, 0.807, 0.96 ]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=0.004670837071489431, scale=1.0278202749916469e-05)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')
QLIKE: AndersonResult(statistic=0.5881416503970591, critical_values=array([0.506, 0.577, 0.692, 0.807, 0.96 ]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=0.04367601527570585, scale=0.00011296891659654825)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')
MAE: AndersonResult(statistic=0.477689647446752

# H=10

In [ ]:
RMSE = []
QLIKE = []
MAE = []
MAPE = []
H=10

for m in range (20):
  randomseed=randrange(129228148)
  model = PatchTST(h=H, input_size=42,
               hidden_size=hidden_size[3], linear_hidden_size =int(hidden_size[3]*2),
             dropout=dropout[0],  fc_dropout=dropout[0], head_dropout=0.3,
                     patch_len=patch_len[1],stride=stride[1],
             attn_dropout=0.3, n_heads=15,
               encoder_layers=2,loss=losses[0],
               max_steps=150,
             learning_rate=learning_rate[1], num_lr_decays=num_lr_decays[1],
               val_check_steps=500, batch_size=32,
               scaler_type='standard',
                random_seed= randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=df1,val_size=H,static_df=df_static,n_windows=None, test_size=test_length-test_length%H,step_size=H)
  forecasts = forecasts.dropna()
  if "PatchTST-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["PatchTST"].values
  else:
    Y_hat=forecasts["PatchTST-median"].values
  Y_true=forecasts["y"].values
  RMSE.append(np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true)))
  QLIKE.append((np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true)))
  MAE.append(np.sum(abs(Y_true-Y_hat))/len(Y_true))
  MAPE.append(np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true))

print(f"RMSE mean: {np.mean(RMSE)} | QLIKE mean: {np.mean(QLIKE)} | MAE mean: {np.mean(MAE)} | MAPE mean: {np.mean(MAPE)}")
print(f"RMSE std: {np.std(RMSE)} | QLIKE std: {np.std(QLIKE)} | MAE std: {np.std(MAE)} | MAPE std: {np.std(MAPE)}")
from scipy.stats import anderson
print(f"RMSE: {anderson(RMSE)}")
print(f"QLIKE: {anderson(QLIKE)}")
print(f"MAE: {anderson(MAE)}")
print(f"MAPE: {anderson(MAPE)}")

df_errors = pd.DataFrame({
    'RMSE': RMSE,
    'QLIKE': QLIKE,
    'MAE': MAE,
    'MAPE': MAPE
})

df_errors.to_excel("PatchTST forecasts.xlsx")

INFO:lightning_fabric.utilities.seed:Seed set to 23521211


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE mean: 0.00549068542322757 | QLIKE mean: 0.05595405913837322 | MAE mean: 0.003382908677894083 | MAPE mean: 0.2576032072715958
RMSE std: 1.2501492483950249e-05 | QLIKE std: 0.00019021232051258142 | MAE std: 7.840852956075302e-06 | MAPE std: 0.0007230715066399105
RMSE: AndersonResult(statistic=0.20134391861196832, critical_values=array([0.506, 0.577, 0.692, 0.807, 0.96 ]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=0.00549068542322757, scale=1.2826260657288216e-05)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')
QLIKE: AndersonResult(statistic=0.252384391935113, critical_values=array([0.506, 0.577, 0.692, 0.807, 0.96 ]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=0.05595405913837322, scale=0.00019515372314579145)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')
MAE: AndersonResult(statistic=0.23612446172994694, 